In [ ]:
from src.utils import data_load
import pandas as pd
from src.s3_utils import pandas_from_csv_s3
import numpy as np
from utils import get_survey_question

In [ ]:
data = data_load(data_keys={'bodyport', 'oura_activity', 'oura_sleep', "surveys"}, wave=5)


In [ ]:
print(len(data['surveys']['user_id'].unique()))
print(len(data['bodyport']['user_id'].unique()))
print(len(data['oura_sleep']['user_id'].unique()))


In [ ]:
df_survey = data['surveys']
survey_question_str = 'swollen'
df_edema = get_survey_question(df_survey, survey_question_str)
df_edema[df_edema['user_id'] == 293][['answer_text', 'date']]

In [ ]:
import datetime
df_birth = data_load(data_keys={'birth'})['birth']
pd.to_datetime(df_birth[df_birth.user_id == 293].birth_date.values[0])
pd.to_datetime(df_birth[df_birth.user_id == 293].birth_date.values[0]) - datetime.timedelta(days=91)

In [ ]:


df_puqe = data['surveys'].loc[data['surveys']['title']=='PUQE']
puqe_user_lst = []
for u in df_puqe['user_id'].unique():
    # 53, 54, 132, 404
    # 404: 'During the past week, did you experience any nausea?'
    # 132: 'On average in a day, for how long do you feel nauseated or sick to your stomach?'
    # 53: 'On average in a day, how many times do you vomit or throw up?'
    # 54: 'On average in a day, how many times have you had retching or dry heaves without bringing anything up?'
    selected_user_puqe = df_puqe[df_puqe['user_id'] == u]
    every_date = selected_user_puqe['date'].unique()
    count = 0
    for d in every_date:
        if len(selected_user_puqe.loc[(selected_user_puqe['date'] == d) & (selected_user_puqe['question_id'] == 404)]):
            if (selected_user_puqe.loc[(selected_user_puqe['date'] == d) & (selected_user_puqe['question_id'] == 404)]['answer_text'].iloc[0]) == 'Yes':
                count += 1
    puqe_user_lst.append((u, count))
most_user = sorted(puqe_user_lst, key=lambda x: x[1], reverse=True)[0][0]

selected_user_puqe = df_puqe[df_puqe['user_id'] == 1032]
selected_user_puqe
sorted_select = sorted(puqe_user_lst, key=lambda x: x[1], reverse=True)
list(map(lambda x: x[0], sorted_select))

# user 1032 has most puqe

In [ ]:
data['bodyport']['subsource'].unique()
# subsource is only `weight`

In [ ]:
data_bodyport = data['bodyport']
users = data_bodyport['user_id'].unique().tolist()
most_data_user = -1
most_data = -1
for u in users:
    num_data = len(data_bodyport[data_bodyport['user_id'] == u])
    if num_data > most_data:
        most_data = num_data
        most_data_user = u
most_data_user
# userid 74 has the most data for bodyport
# userid 293 has the most data for daily surveys

In [ ]:
data_bodyport.columns.tolist()

In [ ]:
selected_data_bodyport = data_bodyport[data_bodyport['user_id'] == 74][['date', 'impedance_ratio', 'peripheral_fluid', 'impedance_mag_1_ohms', 'impedance_phase_1_degs', 'weight_kg']].groupby("date", as_index = False).mean()


In [ ]:
data_oura_activity = data['oura_activity']
data_oura_activity[data_oura_activity['user_id'] == 74]
data_oura_sleep = data['oura_sleep']
# target_user_sleep = data_oura_sleep[data_oura_sleep['user_id'] == 74]
# target_user_sleep[['hr_5min', 'date']]

In [ ]:
data_oura_activity.columns.tolist()

In [ ]:
selected_data_oura_activity = data_oura_activity[data_oura_activity['user_id'] == 74][[
 'cal_active',
 'cal_total',
 'daily_movement',
 'high',
 'inactive',
#  'inactivity_alerts',
 'low',
 'medium',
 'met_min_high',
 'met_min_inactive',
 'met_min_low',
 'met_min_medium',
 'non_wear',
 'rest',
#  'rest_mode_state',
#  'score',
#  'score_meet_daily_targets',
#  'score_move_every_hour',
#  'score_recovery_time',
#  'score_stay_active',
#  'score_training_frequency',
#  'score_training_volume',
 'steps',
 'date']]

In [ ]:
def get_min_date(df):
    return np.min(df['date'])
def get_max_date(df):
    return np.max(df['date'])

In [ ]:

# # union
# overall_min_date = np.min([get_min_date(selected_data_bodyport), get_min_date(selected_data_oura_activity)])
# overall_max_date = np.max([get_max_date(selected_data_bodyport), get_max_date(selected_data_oura_activity)])

# intercept
overall_min_date = np.max([get_min_date(selected_data_bodyport), get_min_date(selected_data_oura_activity)])
overall_max_date = np.min([get_max_date(selected_data_bodyport), get_max_date(selected_data_oura_activity)])

date_range = pd.date_range(overall_min_date, overall_max_date, freq='d')
overall_df = pd.DataFrame()
overall_df['date'] = date_range
selected_data_bodyport['date'] = selected_data_bodyport['date'].astype('datetime64')
selected_data_oura_activity['date'] = selected_data_oura_activity['date'].astype('datetime64')
overall_df = pd.merge(overall_df, selected_data_bodyport, how='left')
overall_df = pd.merge(overall_df, selected_data_oura_activity, how='left')


In [ ]:
np.sum(overall_df.isna())

In [ ]:
overall_df.describe()

In [ ]:
overall_df.loc[overall_df['impedance_ratio'] > 10,'impedance_ratio'] = np.nan
np.sum(overall_df.isna())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style='darkgrid')
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.scatterplot(data=overall_df, x="date", y="non_wear")



In [ ]:
overall_df_filled = overall_df.fillna(method="ffill")
#sns.scatterplot(data=overall_df_filled, x="date", y="non_wear")
sns.scatterplot(x=overall_df_filled["date"], y=overall_df_filled["non_wear"])

In [ ]:
overall_df_filled = overall_df_filled.drop('date', axis=1)
overall_df_filled.to_csv('/mnt/results/user_72_activity_bodyport_ffilled.csv', index=False, header=False)
overall_df_without_date = overall_df.drop('date', axis=1)
overall_df_without_date.to_csv('/mnt/results/user_72_activity_bodyport_unfilled.csv', index=False, header=False)

In [ ]:
overall_df_filled.columns.to_list()

In [ ]:
len(overall_df_filled.columns.to_list())

In [ ]:
np.sum(overall_df_filled.isna())

In [ ]:
len(overall_df_filled)

In [ ]:
# hyperimpute requires python 3.7
from hyperimpute.plugins.imputers import Imputers
import numpy as np
imputers = Imputers()

imputers.list()
method = 'hyperimpute'
plugin = Imputers().get(method)
X = overall_df_without_date
out = plugin.fit_transform(X.copy())

print(method, out)

In [ ]:
hyperimputed_df_without_date = out
hyperimputed_df_without_date.to_csv('/mnt/results/user_72_activity_bodyport_hyperimpute.csv', index=False, header=False)

In [ ]:
hyperimputed_df_without_date['date'] = date_range

In [ ]:
sns.scatterplot(x=hyperimputed_df_without_date["date"], y=hyperimputed_df_without_date["impedance_ratio"], hue=overall_df['impedance_ratio'].isna())